In [21]:
from keras import layers
from keras import models
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
import keras.backend as K
import librosa
import librosa.display
import pylab
import matplotlib.pyplot as plt
from matplotlib import figure
import gc
from path import Path
import numpy as np
from memory_profiler import memory_usage
import os
import pandas as pd
from glob import glob
import numpy as np
from keras_preprocessing.image import ImageDataGenerator

In [22]:
train=pd.read_csv('Train_song/train.csv',dtype=str)
test=pd.read_csv('Test_song/test.csv',dtype=str)

train['ID'] = train['ID']+'.jpg'
test['ID'] = test['ID']+'.jpg'




In [24]:
img_datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)


train_generator=img_datagen.flow_from_dataframe(
    dataframe=train,
    directory="Train_song/Specto",
    x_col="ID",
    y_col="Class",
    subset="training",
    batch_size=32,
    seed=48,
    shuffle=True,
    class_mode="categorical",
    target_size=(64,64))

valid_generator=img_datagen.flow_from_dataframe(
    dataframe=train,
    directory="Train_song/Specto",
    x_col="ID",
    y_col="Class",
    subset="validation",
    batch_size=32,
    seed=48,
    shuffle=True,
    class_mode="categorical",
    target_size=(64,64))

Found 4077 validated image filenames belonging to 10 classes.
Found 1358 validated image filenames belonging to 10 classes.


In [35]:
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

model = Sequential()
model.add(Conv2D(32, (3, 3), activation = 'relu',input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))


model.add(GlobalAveragePooling2D())
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_55 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_55 (Dropout)         (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_48 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_56 (Dropout)         (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 12, 12, 128)     

In [36]:
#Fitting keras model, no test gen for now
train_step=train_generator.n//train_generator.batch_size
valid_step=valid_generator.n//valid_generator.batch_size
#STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=train_step,
                    validation_data=valid_generator,
                    validation_steps=valid_step,
                    epochs=200
)
model.evaluate_generator(generator=valid_generator, steps=valid_step)

Epoch 1/200
127/127 [==============================] - 20s 158ms/step - loss: 1.9414 - accuracy: 0.2710 - val_loss: 1.7962 - val_accuracy: 0.3996
Epoch 2/200
127/127 [==============================] - 22s 170ms/step - loss: 1.4933 - accuracy: 0.4643 - val_loss: 1.2467 - val_accuracy: 0.4925
Epoch 3/200
127/127 [==============================] - 21s 163ms/step - loss: 1.2837 - accuracy: 0.5491 - val_loss: 1.3895 - val_accuracy: 0.5739
Epoch 4/200
127/127 [==============================] - 21s 166ms/step - loss: 1.1151 - accuracy: 0.6222 - val_loss: 1.0693 - val_accuracy: 0.6018
Epoch 5/200
127/127 [==============================] - 23s 183ms/step - loss: 0.9641 - accuracy: 0.6752 - val_loss: 0.8110 - val_accuracy: 0.6968
Epoch 6/200
127/127 [==============================] - 24s 185ms/step - loss: 0.8375 - accuracy: 0.7283 - val_loss: 0.8076 - val_accuracy: 0.6825
Epoch 7/200
127/127 [==============================] - 22s 177ms/step - loss: 0.7822 - accuracy: 0.7402 - val_loss: 0.5759 -

127/127 [==============================] - 28s 219ms/step - loss: 0.0653 - accuracy: 0.9759 - val_loss: 0.9161 - val_accuracy: 0.8861
Epoch 57/200
127/127 [==============================] - 26s 206ms/step - loss: 0.0688 - accuracy: 0.9773 - val_loss: 0.5588 - val_accuracy: 0.8884
Epoch 58/200
127/127 [==============================] - 26s 204ms/step - loss: 0.0858 - accuracy: 0.9718 - val_loss: 0.5907 - val_accuracy: 0.8854
Epoch 59/200
127/127 [==============================] - 27s 210ms/step - loss: 0.0766 - accuracy: 0.9760 - val_loss: 0.5819 - val_accuracy: 0.8808
Epoch 60/200
127/127 [==============================] - 28s 222ms/step - loss: 0.0776 - accuracy: 0.9735 - val_loss: 0.3409 - val_accuracy: 0.8741
Epoch 61/200
127/127 [==============================] - 29s 228ms/step - loss: 0.0816 - accuracy: 0.9711 - val_loss: 0.2984 - val_accuracy: 0.9005
Epoch 62/200
127/127 [==============================] - 28s 224ms/step - loss: 0.1101 - accuracy: 0.9679 - val_loss: 0.0353 - val_a

Epoch 112/200
127/127 [==============================] - 30s 236ms/step - loss: 0.0467 - accuracy: 0.9844 - val_loss: 0.0823 - val_accuracy: 0.8839
Epoch 113/200
127/127 [==============================] - 26s 208ms/step - loss: 0.0384 - accuracy: 0.9871 - val_loss: 0.2962 - val_accuracy: 0.8997
Epoch 114/200
127/127 [==============================] - 26s 205ms/step - loss: 0.0348 - accuracy: 0.9867 - val_loss: 0.0305 - val_accuracy: 0.8741
Epoch 115/200
127/127 [==============================] - 27s 214ms/step - loss: 0.0535 - accuracy: 0.9839 - val_loss: 0.4443 - val_accuracy: 0.8929
Epoch 116/200
127/127 [==============================] - 27s 209ms/step - loss: 0.0703 - accuracy: 0.9754 - val_loss: 1.7737 - val_accuracy: 0.8831
Epoch 117/200
127/127 [==============================] - 28s 223ms/step - loss: 0.0651 - accuracy: 0.9824 - val_loss: 1.6893 - val_accuracy: 0.8673
Epoch 118/200
127/127 [==============================] - 26s 203ms/step - loss: 0.0470 - accuracy: 0.9817 - val_

127/127 [==============================] - 27s 214ms/step - loss: 0.0342 - accuracy: 0.9902 - val_loss: 1.1988 - val_accuracy: 0.8884
Epoch 168/200
127/127 [==============================] - 25s 200ms/step - loss: 0.0247 - accuracy: 0.9928 - val_loss: 0.9094 - val_accuracy: 0.9005
Epoch 169/200
127/127 [==============================] - 26s 201ms/step - loss: 0.0389 - accuracy: 0.9893 - val_loss: 0.6203 - val_accuracy: 0.8922
Epoch 170/200
127/127 [==============================] - 26s 206ms/step - loss: 0.0427 - accuracy: 0.9869 - val_loss: 0.6812 - val_accuracy: 0.8982
Epoch 171/200
127/127 [==============================] - 27s 210ms/step - loss: 0.0324 - accuracy: 0.9894 - val_loss: 0.2665 - val_accuracy: 0.8876
Epoch 172/200
127/127 [==============================] - 27s 213ms/step - loss: 0.0327 - accuracy: 0.9902 - val_loss: 2.4901 - val_accuracy: 0.8824
Epoch 173/200
127/127 [==============================] - 26s 201ms/step - loss: 0.0331 - accuracy: 0.9891 - val_loss: 1.1449 -

[0.40851712226867676, 0.8958333134651184]

In [50]:
test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
    dataframe=test,
    directory="Test_song/Specto",
    x_col="ID",
    y_col=None,
    batch_size=32,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(64,64))
test_step=test_generator.n//test_generator.batch_size

Found 3297 validated image filenames.


In [51]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=test_step,
verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)

#Fetch labels from train gen for testing
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
print(predictions[0:20])

103/103 [==============================] - 7s 65ms/step
['drilling', 'dog_bark', 'drilling', 'dog_bark', 'street_music', 'jackhammer', 'jackhammer', 'children_playing', 'dog_bark', 'siren', 'dog_bark', 'siren', 'dog_bark', 'children_playing', 'street_music', 'siren', 'dog_bark', 'drilling', 'drilling', 'jackhammer']


In [39]:
# for saving the model
model.save("mods.h5")

In [49]:
from numpy import loadtxt
from keras.models import load_model
 
# load model
model = load_model('mods.h5')